In [ ]:
## import public packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import mlab
from datetime import datetime
from sklearn.cluster import KMeans
from scipy.stats import norm
from scipy.stats import multivariate_normal
# from __future__ import 

## import self-written packages 

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
def pi1_gen(X):
    pi1=0.1*norm.pdf(X, loc=0.5, scale=0.1)+0.1*norm.pdf(X, loc=-0.8, scale=0.1)
    pi1+=0.1*(X+1) 
    return pi1
def datagen_1D_continuous(job=0, n_samples=10000,data_vis=0, num_case=4):
    if job == 0: # Gaussian mixtures 
        X = np.random.uniform(-1,1,size = n_samples)
        pi1=pi1_gen(X)
        p = np.zeros(n_samples)
        h = np.zeros(n_samples)
               
        for i in range(n_samples):
            rnd = np.random.uniform()
            if rnd > pi1[i]:
                p[i] = np.random.uniform()
                h[i] = 0
            else:
                p[i] = np.random.beta(a = 0.3, b = 4)
                h[i] = 1
        
        if data_vis == 1:
            fig = plt.figure()
            ax1 = fig.add_subplot(121)
            x_grid = np.arange(-1, 1, 1/100.0)
            pi1_grid = pi1_gen(x_grid)  
            ax1.plot(x_grid, pi1_grid)
            
            ax2 = fig.add_subplot(122)
            alt=ax2.scatter(X[h==1][1:50], p[h==1][1:50],color='r')
            nul=ax2.scatter(X[h==0][1:50], p[h==0][1:50],color='b')
#             ax2.legend((alt,nul),('alternatives', 'nulls'))
            plt.show()
            
        return p, h, np.reshape(X,[n_samples,1])
def Storey_BH(x, alpha = 0.05, lamb=0.6):
    pi0_hat=np.divide(np.sum(x>lamb),x.shape[0] *(1-lamb))
    alpha /= pi0_hat
    x_s = sorted(x)
    n = len(x_s)
    ic = 0
    for i in range(n):
        if x_s[i] < i*alpha/float(n):
            ic = i
    return ic, x_s[ic], pi0_hat
